<a href="https://colab.research.google.com/github/Code-sHimanshu/Collect-your-GamingTools/blob/main/Sentiment%20Analysis%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount the drive


**Load all the three datasets**

In [ ]:
import pandas as pd

isear_df = pd.read_csv("path_to/isear.csv")  # or isear_dataset.csv
isear_df = isear_df[['emotion', 'sentence']]  # rename columns if necessary
isear_df.columns = ['label', 'text']
isear_df['source'] = 'ISEAR'


**Load ISEAR Dataset (Kaggle CSV)**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("faisalsanto007/isear-dataset")

print("Path to dataset files:", path)

**Load GoEmotions Dataset (from HuggingFace)**

In [ ]:
from datasets import load_dataset

goemotions = load_dataset("go_emotions", split='train')  # you can load 'validation' and 'test' too
goemotions_df = goemotions.to_pandas()

# You may need to convert numeric labels to actual emotions
from datasets import ClassLabel
emotions = goemotions.features['labels'].feature.names

# Map first label (multi-label, use first only or join them)
goemotions_df['label'] = goemotions_df['labels'].apply(lambda x: emotions[x[0]] if len(x) > 0 else 'neutral')
goemotions_df['text'] = goemotions_df['text']
goemotions_df['source'] = 'GoEmotions'
goemotions_df = goemotions_df[['label', 'text', 'source']]


**Load Amazin Reviews Multi Datast (from HuggingFace)**

In [ ]:
amazon = load_dataset("amazon_reviews_multi", "en", split='train')  # You can load other languages
amazon_df = amazon.to_pandas()

# Convert star ratings into sentiment labels
def convert_rating_to_sentiment(rating):
    if rating <= 2:
        return "negative"
    elif rating == 3:
        return "neutral"
    else:
        return "positive"

amazon_df['label'] = amazon_df['stars'].apply(convert_rating_to_sentiment)
amazon_df['text'] = amazon_df['review_body']
amazon_df['source'] = 'Amazon'
amazon_df = amazon_df[['label', 'text', 'source']]


**Preprocess and combine the Datasets**

In [ ]:
# Ensure all datasets have the same structure
combined_df = pd.concat([isear_df, goemotions_df, amazon_df], ignore_index=True)

# Optional: Clean text
import re

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

combined_df['text'] = combined_df['text'].apply(clean_text)

# Save to a CSV file
combined_df.to_csv("combined_emotion_sentiment_dataset.csv", index=False)
